# Getting Started

Pandas-TD allows you to get your query result as a DataFrame and to store DataFrame to Treasure Data.

Before starting any session, we recommend to set your API key to the environment variable ``TD_API_KEY``:

    export TD_API_KEY=1234/abcd...

You can initialize your connection by ``pandas_td.connect``:

In [1]:
import os
import numpy as np
import pandas as pd
from pandas_td import connect
from pandas_td import read_td
from pandas_td import to_td

# Initialize connection
td = connect(apikey=os.environ['TD_API_KEY'], endpoint='https://api.treasuredata.com')

Using a connection, you can use some convenient methods like ``databases`` and ``tables``:

In [2]:
# Get a list of databases
td.databases()

,name,count,permission,created_at,updated_at
0,tutorial,0,full_access,2015-05-27 13:12:43+00:00,2015-05-27 13:12:43+00:00
1,sample_datasets,8812278,query_only,2014-10-04 01:13:11+00:00,2015-01-09 01:07:14+00:00


In [3]:
# Get a list of tables
td.tables('sample_datasets')

,name,count,estimated_storage_size,last_log_timestamp,created_at
0,nasdaq,8807278,168205061,1970-01-01 00:00:00+00:00,2014-10-08 02:57:38+00:00
1,www_access,5000,0,2014-10-04 01:13:15+00:00,2014-10-04 01:13:12+00:00


You can choose a query engine and pass it to ``pandas_td.read_td`` to run a query:

In [4]:
# Select a query engine
presto = td.query_engine(database='sample_datasets', type='presto')

# Run a query
read_td('select * from nasdaq limit 5', presto)

,symbol,open,volume,high,low,close,time
0,ACGL,54.45,842365,54.83,54.450,54.70,1411401600
1,APEI,28.59,132661,28.65,27.790,28.28,1411401600
2,AVEO,1.29,461162,1.29,1.155,1.18,1411401600
3,AMSWA,8.79,79824,8.89,8.720,8.74,1411401600
4,AROW,25.92,16818,26.00,25.530,25.54,1411401600


It is also straightforward to import a DataFrame into a table by using ``pandas_td.to_td``:

In [5]:
# Create a DataFrame with random values
df = pd.DataFrame(np.random.rand(4, 3), columns=['x', 'y', 'z'])

# Import it into 'tutorial.tut1'
to_td(df, 'tutorial.tut1', td, index=False)

In [6]:
# Check the result
read_td('select * from tutorial.tut1', presto)

,z,y,x,time
0,0.898377,0.053566,0.101563,1432734481
1,0.670632,0.503762,0.566276,1432734481
2,0.139800,0.210144,0.295164,1432734481
3,0.535834,0.768914,0.361069,1432734481
